In [ ]:
#pip install citypy

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from citipy import citipy

import requests

from config import weather_api_key

import time
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=11)
lngs = np.random.uniform(low=-180.000, high=180.000, size=11)
# zip method
lat_lngs = zip(lats, lngs)
lat_lngs

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [3]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

11

In [4]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
# print(url)

In [5]:
# Make a request

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# print (requests.get(city_url).json())

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | airai
Processing Record 2 of Set 1 | namibe
Processing Record 3 of Set 1 | santarem
Processing Record 4 of Set 1 | carnarvon
Processing Record 5 of Set 1 | nizhneivkino
Processing Record 6 of Set 1 | owando
Processing Record 7 of Set 1 | pucallpa
Processing Record 8 of Set 1 | port augusta
Processing Record 9 of Set 1 | poum
Processing Record 10 of Set 1 | viransehir
Processing Record 11 of Set 1 | busselton
-----------------------------
Data Retrieval Complete      
-----------------------------


In [7]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
# New order
new_column_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Airai,TL,2022-08-11 21:40:30,-8.9266,125.4092,57.07,93,51,3.53
1,Namibe,AO,2022-08-11 21:40:31,-15.1961,12.1522,61.38,90,54,8.57
2,Santarem,BR,2022-08-11 21:40:31,-2.4431,-54.7083,87.93,70,20,8.05
3,Carnarvon,AU,2022-08-11 21:40:32,-24.8667,113.6333,58.78,34,0,15.77
4,Nizhneivkino,RU,2022-08-11 21:40:32,58.1940,49.5214,54.64,85,81,6.02
5,Owando,CG,2022-08-11 21:40:32,-0.4819,15.8999,77.95,48,99,1.30
6,Pucallpa,PE,2022-08-11 21:37:14,-8.3791,-74.5539,86.04,54,40,4.61
7,Port Augusta,AU,2022-08-11 21:40:33,-32.5000,137.7667,47.66,93,35,4.72
8,Poum,NC,2022-08-11 21:40:33,-20.2333,164.0167,68.79,60,0,7.87
9,Viransehir,TR,2022-08-11 21:40:34,37.2353,39.7631,82.22,57,0,8.10


In [8]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")